# Clustering Destinations
## Applied Data Science course - Capstone project

### Installing necessary libraries

In [26]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import html


import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
!pip install foursquare
import foursquare

In [5]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

### Parsing city data from wikipedia

In [15]:
WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors'

cities = pd.DataFrame(columns=['City', 'Country'])

soup = BeautifulSoup(requests.get(WIKI_URL).text, 'html.parser')
for row in soup.select_one('table.wikitable tbody').find_all('tr'):
    cols = row.find_all('td')
    if len(cols) is 0:
        continue
    cities = cities.append({'City': cols[2].get_text().strip(), 'Country': cols[3].get_text().strip()}, ignore_index=True)



In [20]:
cities.shape

(139, 2)

### Gathering location data for the cities

In [8]:
MAX_RETRIES = 10
def get_lat_lon(city, country):
    
    retries = MAX_RETRIES
    location = None
    while retries > 0 and location is None:
        location = geocoder.osm('{}, {}'.format(city, country)).latlng
        retries -= 1
        
    if location is None:        
        return { 'Latitude': None, 'Longitude': None }

    return { 'Latitude': location[0], 'Longitude': location[1] }



In [21]:
lat_lon_df = pd.DataFrame(columns=['Latitude', 'Longitude'])
for i, row in cities.iterrows():
    lat_lon_df = lat_lon_df.append(get_lat_lon(row['City'], row['Country']), ignore_index=True)
cities_with_location = pd.concat([cities, lat_lon_df], axis=1, sort=False)

In [29]:
cities_with_location.shape

(139, 4)

In [28]:
cities_with_location.head()

,City,Country,Latitude,Longitude
0,Hong Kong,Hong Kong,22.279328,114.162813
1,Bangkok,Thailand,13.754253,100.493087
2,London,United Kingdom,51.507322,-0.127647
3,Macau,Macau,-5.113366,-36.634996
4,Singapore,Singapore,1.357107,103.819499


In [24]:
cities_with_location.dropna(inplace=True)

In [30]:
cities_with_location.shape

(139, 4)

### Visualizing cities

In [36]:
import html

world_map = folium.Map(location=[0, 0], zoom_start=2)

for lat, lon, city, country in zip(cities_with_location.Latitude, cities_with_location.Longitude, cities_with_location.City, cities_with_location.Country):
    folium.features.Marker(
        [lat, lon],
        popup=folium.Popup('<h3>{}</h3><h4>{}</h4>'.format(html.escape(city), html.escape(country))),
    ).add_to(world_map)

world_map

### Getting venue data for the cities

In [37]:
# The code was removed by Watson Studio for sharing.

In [38]:
fs_client = foursquare.Foursquare(client_id=FOURSQUARE_CLIENT_ID, client_secret=FOURSQUARE_CLIENT_SECRET)

In [51]:
def get_venues(city, country, lat, lon):
    try:
        venues = pd.DataFrame(columns=['City', 'Country', 'Name', 'Category', 'Latitude', 'Longitude', 'City_Latitude', 'City_Longitude'])
        offset = 0
        limit = 50
        while True:
            response = fs_client.venues.explore(params={'near': '{}, {}'.format(city, country), 'time': 'any', 'day': 'any', 'radius': 10000, 'limit': limit, 'offset': offset})
            totalResults = min(response['totalResults'], 250)
            print("Getting venues for", city, country, offset, limit, totalResults)
            for v in response['groups'][0]['items']:
                venues = venues.append({
                    'City': city,
                    'Country': country,
                    'Name': v['venue']['name'],
                    'Category': v['venue']['categories'][0]['pluralName'],
                    'Latitude': v['venue']['location']['lat'],
                    'Longitude': v['venue']['location']['lng'],
                    'City_Latitude': lat,
                    'City_Longitude': lon
                }, ignore_index=True)
            offset += limit
            if offset > totalResults:
                break
    except:
        print('An error occured for', city, country)
    return venues

In [52]:
venues_with_cities = pd.DataFrame(columns=['City', 'Country', 'Name', 'Category', 'Latitude', 'Longitude', 'City_Latitude', 'City_Longitude'])

for lat, lon, city, country in zip(cities_with_location.Latitude, cities_with_location.Longitude, cities_with_location.City, cities_with_location.Country):
    venues_with_cities = pd.concat([venues_with_cities, get_venues(city, country, lat, lon)], axis=0)

venues_with_cities.head()


Getting venues for Hong Kong Hong Kong 0 50 238
Getting venues for Hong Kong Hong Kong 50 50 238
Getting venues for Hong Kong Hong Kong 100 50 238
Getting venues for Hong Kong Hong Kong 150 50 238
Getting venues for Hong Kong Hong Kong 200 50 238
Getting venues for Bangkok Thailand 0 50 213
Getting venues for Bangkok Thailand 50 50 213
Getting venues for Bangkok Thailand 100 50 213
Getting venues for Bangkok Thailand 150 50 213
Getting venues for Bangkok Thailand 200 50 213
Getting venues for London United Kingdom 0 50 238
Getting venues for London United Kingdom 50 50 238
Getting venues for London United Kingdom 100 50 238
Getting venues for London United Kingdom 150 50 238
Getting venues for London United Kingdom 200 50 238
Getting venues for Macau Macau 0 50 29
Getting venues for Singapore Singapore 0 50 237
Getting venues for Singapore Singapore 50 50 237
Getting venues for Singapore Singapore 100 50 237
Getting venues for Singapore Singapore 150 50 237
Getting venues for Singapore

,City,Country,Name,Category,Latitude,Longitude,City_Latitude,City_Longitude
0,Hong Kong,Hong Kong,Four Seasons Hotel Hong Kong (香港四季酒店),Hotels,22.286554,114.156929,22.279328,114.162813
1,Hong Kong,Hong Kong,Mandarin Grill + Bar (文華扒房＋酒吧),Steakhouses,22.281928,114.159408,22.279328,114.162813
2,Hong Kong,Hong Kong,Pure Yoga,Yoga Studios,22.283022,114.155674,22.279328,114.162813
3,Hong Kong,Hong Kong,Mandarin Oriental Hong Kong (香港文華東方酒店),Hotels,22.281857,114.159382,22.279328,114.162813
4,Hong Kong,Hong Kong,Samsen (泰館),Thai Restaurants,22.285082,114.152286,22.279328,114.162813


In [53]:
venues_with_cities.groupby('City').count()

,Country,Name,Category,Latitude,Longitude,City_Latitude,City_Longitude
City,,,,,,,
Abu Dhabi,219,219,219,219,219,219,219
Accra,134,134,134,134,134,134,134
Agra,62,62,62,62,62,62,62
Amman,192,192,192,192,192,192,192
Amsterdam,239,239,239,239,239,239,239
Antalya,210,210,210,210,210,210,210
Athens,237,237,237,237,237,237,237
Atlanta,216,216,216,216,216,216,216
Auckland,229,229,229,229,229,229,229
